
Based on https://chagall.med.cornell.edu/RNASEQcourse/Intro2RNAseq.pdf


Make plots are wider.

In [ ]:
options(repr.plot.width=16, repr.plot.height=9)

In [ ]:
#featureCounts='/francislab/data1/raw/20191008_Stanford71/trimmed/unpaired/subread-rna.hsa.featureCounts.miRNA_primary_transcript.csv'
#metadata='/francislab/data1/raw/20191008_Stanford71/metadata.csv'
featureCounts=Sys.getenv('featureCounts')
metadata=Sys.getenv('metadata')

In [ ]:
print(featureCounts)

In [ ]:
print(metadata)

In [ ]:
countdata <- read.table(featureCounts, header=TRUE, row.names=1, sep='	')
head(countdata)

Remove first five columns (chr, start, end, strand, length)
or df = subset(df, select=-c(Chr,Start,End,Strand,Length))

In [ ]:
countdata <- countdata[ ,6:ncol(countdata)]
head(countdata)

In [ ]:
countdata <- as.matrix(countdata)
head(countdata)

In [ ]:
nrow(countdata)

In [ ]:
casecontrol <- read.table(metadata, header=TRUE, row.names=1, sep=',')
head(casecontrol)

expecting comma separated, 2 column csf with id and cc columns

In [ ]:
condition <- casecontrol$cc
head(condition)

Analysis with DESeq2

In [ ]:
suppressMessages({
  library(DESeq2)
})

Create a coldata frame and instantiate the DESeqDataSet. See ?DESeqDataSetFromMatrix

In [ ]:
# Why wrapped in ( parentheses )?
(coldata <- data.frame(row.names=colnames(countdata), condition))
head(coldata)

In [ ]:
dds <- DESeqDataSetFromMatrix(countData=countdata, colData=coldata, design=~condition)
dds

Run the DESeq pipeline

In [ ]:
dds <- DESeq(dds)

In [ ]:
dds

From Intro to DGE Analysis remove genes without any counts

In [ ]:
dds <- dds[rowSums(counts(dds)) > 0,]
dds

NORMALIZE?

In [ ]:
# don't think that this needs to be explicitly called
#dds <- estimateSizeFactors(dds)

In [ ]:
dds

In [ ]:
head(data.frame(sizeFactors(dds)))

In [ ]:
counts.sf_normalized <- counts(dds, normalized=TRUE)
head(counts.sf_normalized)

In [ ]:
log.norm.counts <- log2(counts.sf_normalized + 1)
head(log.norm.counts)

In [ ]:
boxplot(counts.sf_normalized, notch=TRUE,
  main='untransformed read counts', ylab='read counts')

In [ ]:
boxplot(log.norm.counts, notch=TRUE,
  main='log2-transformed read counts',
  ylab='log2(read counts)')

In [ ]:
plot(log.norm.counts[,1:2], cex=.1, main='Normalized log2(read counts)')

In [ ]:
library ( vsn )
library ( ggplot2 )

In [ ]:
msd_plot<-meanSdPlot(log.norm.counts,ranks=FALSE,plot=FALSE)
msd_plot$gg+ggtitle('sequencing depth normalized log2(read counts)')+ylab('standard deviation')

In [ ]:
dds.rlog<-rlog(dds,blind=TRUE)
rlog.norm.counts<-assay(dds.rlog)

In [ ]:
msd_plot<-meanSdPlot(rlog.norm.counts,ranks=FALSE,plot=FALSE)
msd_plot$gg+ggtitle('rlog-transformed read counts')+ylab('standard deviation')
# cor () calculates the correlation between columns of a matrix

In [ ]:
distance.m_rlog<-as.dist(1-cor(rlog.norm.counts,method='pearson'))
head(distance.m_rlog)

In [ ]:
plot(hclust(distance.m_rlog),
  labels=colnames(rlog.norm.counts),
  main='rlog transformed read counts distance : Pearson correlation')

In [ ]:
pc<-prcomp(t(rlog.norm.counts))

In [ ]:
plot(pc$x[,1],pc$x[,2],
  col=colData(dds)[,1],
  main='PCA of seq.depth normalized and rlog-transformed read counts')

In [ ]:
P<-plotPCA(dds.rlog)
P<-P+theme_bw()+ggtitle('Rlog transformed counts')
P

In [ ]:
res <- results(dds)
head(results(dds, tidy=TRUE))

In [ ]:
summary(res) #summary of results

In [ ]:
res[res$pvalue < 0.05,]

In [ ]:
res[res$pvalue < 0.1,]

In [ ]:
plotMA( res, ylim = c(-1, 1) )

In [ ]:
plotMA( res, ylim = c(-2, 2) )

In [ ]:
plotMA( res, ylim = c(-5, 5) )

In [ ]:
plotDispEsts( dds, ylim = c(1e-6, 1e1) )

In [ ]:
plotDispEsts( dds, ylim = c(1e-6, 1e4) )

In [ ]:
hist( res$pvalue, breaks=20, col='grey' )

In [ ]:
hist( res$padj, breaks=20, col='grey' )

In [ ]:
res <- res[order(res$padj),]
head(res)

In [ ]:
print('Looping over head 10 padj')
for(gene in row.names(head(res[order(res$padj),],10))){
  plotCounts(dds, gene=gene, intgroup='condition',
    main=paste(gene,' - pval',res[gene,]$pvalue, 'padj',res[gene,]$padj, sep=' : '))
}
print('end loop over head 10')

In [ ]:
print('Looping over head 10 pvalue')
for(gene in row.names(head(res[order(res$pvalue),],10))){
  plotCounts(dds, gene=gene, intgroup='condition',
    main=paste(gene,' - pval',res[gene,]$pvalue, 'padj',res[gene,]$padj, sep=' : '))
}
print('end loop over head 10')

Make a basic volcano plot

In [ ]:
with(res, plot(log2FoldChange, -log10(pvalue), pch=20, main='Volcano plot', xlim=c(-3,3)))

In [ ]:
with(res, plot(log2FoldChange, -log10(pvalue), pch=20, main='Volcano plot', xlim=c(-3,3)))
with(subset(res, padj<.01 ), points(log2FoldChange, -log10(pvalue), pch=20, col='blue'))
with(subset(res, padj<.01 & abs(log2FoldChange)>2), points(log2FoldChange, -log10(pvalue), pch=20, col='red'))
with(subset(res, padj<.001 ), points(log2FoldChange, -log10(pvalue), pch=20, col='green'))
with(subset(res, padj<.001 & abs(log2FoldChange)>2), points(log2FoldChange, -log10(pvalue), pch=20, col='yellow'))
legend('topright',
  legend=c('All','padj<0.01','padj<0.01 & abs>2','padj<0.001','padj<0.001 & abs>2'),
  col=c('black','blue','red','green','yellow'),
  pch = 20,
  lty=1:2, cex=0.8)

In [ ]:
rld <- rlog(dds)

In [ ]:
plotPCA(rld, intgroup=c( 'condition' ))

also possible to perform custom transformation:
estimateSizeFactors

In [ ]:
dds

In [ ]:
# don't think that this needs to be explicitly called
#dds <- estimateSizeFactors(dds) # why do I keep running this? (too many cut-n-paste tutorials)

In [ ]:
dds

shifted log of normalized counts
SummarizedExperiment

In [ ]:
se <- SummarizedExperiment(log2(counts(dds, normalized=TRUE) + 1), colData=colData(dds))

the call to DESeqTransform() is needed to
trigger our plotPCA method.
plotPCA

In [ ]:
plotPCA( DESeqTransform( se ), intgroup=c( 'condition' ) )

Concentration of counts over total sum of counts
estimateSizeFactors

In [ ]:
# don't think that this needs to be explicitly called
#dds <- estimateSizeFactors(dds) # why do I keep running this?

In [ ]:
plotSparsity(dds)

Cluster Dendrogram
varianceStabilizingTransformation

In [ ]:
vsd <- varianceStabilizingTransformation(dds)

In [ ]:
dists <- dist(t(assay(vsd)))

In [ ]:
head(dists)

In [ ]:
plot(hclust(dists))

In [ ]:
plotDispEsts(dds, main='Dispersion plot')

Regularized log transformation for clustering/heatmaps, etc

In [ ]:
rld <- rlogTransformation(dds)
head(assay(rld))

In [ ]:
hist(assay(rld))
# Colors for plots below
## Ugly:
## (mycols <- 1:length(unique(condition)))
## Use RColorBrewer, better

In [ ]:
library(RColorBrewer)
# Why wrapped in (parentheses)?
(mycols <- brewer.pal(8, 'Dark2')[1:length(unique(condition))])

Sample distance heatmap

In [ ]:
sampleDists <- as.matrix(dist(t(assay(rld))))
library(gplots)

In [ ]:
heatmap.2(as.matrix(sampleDists), key=F, trace='none',
  col=colorpanel(100, 'black', 'white'),
  ColSideColors=mycols[condition], RowSideColors=mycols[condition],
  margin=c(10, 10), main='Sample Distance Matrix')

Principal components analysis
Could do with built-in DESeq2 function:
DESeq2::plotPCA(rld, intgroup='condition')
I like mine better:
define rld_pca

In [ ]:
rld_pca <- function (rld, intgroup = 'condition', ntop = 500, colors=NULL,
    legendpos='bottomleft', main='PCA Biplot', textcx=1, ...) {
  require(genefilter)
  require(calibrate)
  require(RColorBrewer)
  rv = rowVars(assay(rld))
  select = order(rv, decreasing = TRUE)[seq_len(min(ntop, length(rv)))]
  pca = prcomp(t(assay(rld)[select, ]))
  fac = factor(apply(as.data.frame(colData(rld)[, intgroup, drop = FALSE]), 1, paste, collapse = ' : '))
  if (is.null(colors)) {
    if (nlevels(fac) >= 3) {
      colors = brewer.pal(nlevels(fac), 'Paired')
    } else {
      colors = c('black', 'red')
    }
  }
  pc1var <- round(summary(pca)$importance[2,1]*100, digits=1)
  pc2var <- round(summary(pca)$importance[2,2]*100, digits=1)
  pc1lab <- paste0('PC1 (',as.character(pc1var),'%)')
  pc2lab <- paste0('PC1 (',as.character(pc2var),'%)')
  plot(PC2~PC1, data=as.data.frame(pca$x), bg=colors[fac], pch=21, xlab=pc1lab, ylab=pc2lab, main=main, ...)
  with(as.data.frame(pca$x), textxy(PC1, PC2, labs=rownames(as.data.frame(pca$x)), cex=textcx))
  legend(legendpos, legend=levels(fac), col=colors, pch=20)
  #     rldyplot(PC2 ~ PC1, groups = fac, data = as.data.frame(pca$rld),
  #            pch = 16, cerld = 2, aspect = 'iso', col = colours, main = draw.key(key = list(rect = list(col = colours),
  #                                                                                         terldt = list(levels(fac)), rep = FALSE)))
}

In [ ]:
rld_pca(rld, colors=mycols, intgroup='condition', xlim=c(-75, 35))

Get differential expression results

In [ ]:
res <- results(dds)
table(res$padj<0.05)

Order by adjusted p-value

In [ ]:
res <- res[order(res$padj), ]

Merge with normalized count data

In [ ]:
resdata <- merge(as.data.frame(res), as.data.frame(counts(dds, normalized=TRUE)), by='row.names', sort=FALSE)

In [ ]:
names(resdata)[1] <- 'Gene'

In [ ]:
head(resdata)

In [ ]:
## Write results
#print('write csv')
#write.csv(resdata, file=paste0(opt$featureCounts,'.deseq.diffexpr-results.csv'))

Examine plot of p-values
hist

In [ ]:
hist(res$pvalue, breaks=50, col='grey')

Examine independent filtering
attr filterThreshold

In [ ]:
attr(res, 'filterThreshold')

MA plot
Could do with built-in DESeq2 function:
DESeq2::plotMA(dds, ylim=c(-1,1), cex=1)
I like mine better:
define mplot

In [ ]:
maplot <- function (res, thresh=0.05, labelsig=TRUE, textcx=1, ...) {
  with(res, plot(baseMean, log2FoldChange, pch=20, cex=.5, log='x', ...))
  with(subset(res, padj<thresh), points(baseMean, log2FoldChange, col='red', pch=20, cex=1.5))
  if (labelsig) {
    require(calibrate)
    with(subset(res, padj<thresh), textxy(baseMean, log2FoldChange, labs=Gene, cex=textcx, col=2))
  }
}

In [ ]:
maplot(resdata, main='MA Plot')

Volcano plot with 'significant' genes labeled
define volcanoplot

In [ ]:
volcanoplot <- function (res, lfcthresh=2, sigthresh=0.05, main='Volcano Plot',
    legendpos='bottomright', labelsig=TRUE, textcx=1, ...) {
  with(res, plot(log2FoldChange, -log10(pvalue), pch=20, main=main, ...))
  with(subset(res, padj<sigthresh ), points(log2FoldChange,
    -log10(pvalue), pch=20, col='red', ...))
  with(subset(res, abs(log2FoldChange)>lfcthresh), points(log2FoldChange,
    -log10(pvalue), pch=20, col='orange', ...))
  with(subset(res, padj<sigthresh & abs(log2FoldChange)>lfcthresh),
    points(log2FoldChange, -log10(pvalue), pch=20, col='green', ...))
  if (labelsig) {
    require(calibrate)
    with(subset(res, padj<sigthresh & abs(log2FoldChange)>lfcthresh),
      textxy(log2FoldChange, -log10(pvalue), labs=Gene, cex=textcx, ...))
  }
  legend(legendpos, xjust=1, yjust=1,
    legend=c(paste('FDR<',sigthresh,sep=''),
    paste('|LogFC|>',lfcthresh,sep=''), 'both'),
    pch=20, col=c('red','orange','green'))
}

In [ ]:
volcanoplot(resdata, lfcthresh=1, sigthresh=0.05, textcx=.8, xlim=c(-2.3, 2))

In [ ]:
# don't think that this needs to be explicitly called
#dds <- estimateSizeFactors ( dds ) # sequencing depth normalization between the samples
#dds <- estimateDispersions ( dds ) # gene - wise dispersion estimates across all samples
dds <- nbinomWaldTest ( dds ) # this fits a negative binomial GLM and applies Wald statistics to each gene

In [ ]:
dds.results <- results( dds , independentFiltering = TRUE , alpha = 0.05)
summary ( dds.results )

In [ ]:
# the DESeqResult object can basically be handled like a data . frame
head ( dds.results )

In [ ]:
table ( dds.results $ padj < 0.05)

In [ ]:
rownames ( subset ( dds.results , padj < 0.05) )

In [ ]:
hist ( dds.results$pvalue ,
  col='grey', border='white', xlab='', ylab='',
  main='frequencies of p-values' )

In [ ]:
# plotMA ( dds.results , alpha = 0.05 , main = "WT vs. SNF2 mutants", ylim = c( -4 ,4) )

In [ ]:
# load the library with the aheatmap () function
library ( NMF )

# aheatmap needs a matrix of values , e.g. , a matrix of DE genes with the transformed read counts for each replicate
# sort the results according to the adjusted p- value
dds.results.sorted <- dds.results [ order ( dds.results$padj ) , ]

# identify genes with the desired adjusted p- value cut -off
DGEgenes <- rownames ( subset ( dds.results.sorted , padj < 0.05) )

# extract the normalized read counts for DE genes into a matrix
hm.mat_DGEgenes <- log.norm.counts [ DGEgenes , ]

# plot the normalized read counts of DE genes sorted by the adjusted p- value
#aheatmap ( hm.mat_DGEgenes , Rowv = NA , Colv = NA )

#Error in dimnames(x) <- dn: length of 'dimnames' [1] not equal to array extent
#Traceback:
#1. aheatmap(hm.mat_DGEgenes, Rowv = NA, Colv = NA)
#2. `rownames<-`(`*tmp*`, value = generate_dimnames(labRow, nrow(mat), 
# .     rownames(mat)))

# combine the heatmap with hierarchical clustering
#aheatmap ( hm.mat_DGEgenes ,
#  Rowv = TRUE , Colv = TRUE , # add dendrograms to rows and columns
#  distfun = "euclidean" , hclustfun = "average" )

# scale the read counts per gene to emphasize the sample -type - specific differences
#aheatmap ( hm.mat_DGEgenes ,
#  Rowv = TRUE , Colv = TRUE ,
#  distfun = "euclidean" , hclustfun = "average" ,
#  scale = "row" )
# values are transformed into distances from the center of the row - specific average : ( actual value - mean of the group ) / standard deviation